In [ ]:
import torch
from torch import nn  # nn contains all of PyTorch's building blocks for neural networks
import matplotlib.pyplot as plt

print(f"GPU Available: {torch.cuda.get_device_name(0)}")

# Check PyTorch version
print("Torch Version", torch.__version__)

### Creating a simple dataset using the linear regression formula

In [ ]:
# Create *known* parameters
weight = 0.7
bias = 0.3

# Create
start = 0
end = 1
step = 0.02

x = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * x + bias

# x[:10], y[:10]

In [ ]:
# len(x), len(y)

### Splitting data into training and test sets

In [ ]:
# Create a train/test split
train_split = int(0.8 * len(x))
x_train, y_train = x[:train_split], y[:train_split]
x_test, y_test = x[train_split:], y[train_split:]
len(x_train), len(y_train), len(x_test), len(y_test)

### Exploring data

In [ ]:
def plot_predictions(
    train_data=x_train,
    train_labels=y_train,
    test_data=x_test,
    test_labels=y_test,
    predictions=None,
):
    """
    Plots training data, test data and compares predictions.
    """
    plt.figure(figsize=(10, 7))

    # Plot training data in blue
    plt.scatter(train_data, train_labels, s=4, c="b", label="Training data")

    # Plot test data in green
    plt.scatter(test_data, test_labels, s=4, c="g", label="Testing data")

    # Are there predictions?
    if predictions is not None:
        # Plot the predictions if they exists
        plt.scatter(test_data, predictions, s=4, c="r", label="Predictions")

    # Show the legend
    plt.legend(prop={"size": 14})

In [ ]:
plot_predictions()

### Building first PyTorch model!
 * start with randon values (weight & bias)
 * look at the training data and adjust the random values to better represent (or get closer to) the ideal values (the weight and bias values we used to create the data)


How does it do so?
Through two main algorithms:
1. Gradient descent
2. Backpropagation

In [ ]:
import torch
from torch import nn

# Create a linerar regression model class
class LinearRegressionModel(
    nn.Module  # <- nn.Module contains all the building blocks for neural network
):  # <- almost everything in PyTorch inherits from nn.Module
    def __init__(self):
        super().__init__()

        # Initalize model parameters
        self.weight = nn.Parameter(
            torch.randn(
                1,  # <- start with a random weight and try to adjust it to the ideal weight
                requires_grad=True,  # <- can this parameter be updated via gradient descent?
                dtype=torch.float32,  # PyTorch loves the datatype torch.float32
            )
        )  # <- PyTorch loves the datatype torch.float32

        self.bias = nn.Parameter(
            torch.randn(
                1,  # <- start with a random bias and try to adjust it to the ideal bias
                requires_grad=True,  # <- can this parameter be updated via gradient descent?
                dtype=torch.float32,  # PyTorch loves the datatype torch.float32
            )
        )

    # Forward method to define the computation in the model, all subclasses of nn.Module need to overwrite forward method.
    # This defines the forward computation of the model
    def forward(self, x: torch.Tensor) -> torch.Tensor:  # <- "x" is the input data
        return self.weight * x + self.bias  # this is the linear regression formula

### PyTorch model building essentials

1. **torch.nn** - contains all of the buildings for computational graphs (a neural network can be considered a computational graph)

2. **torch.nn.Parameter** - what parameters should our model try and learn, often a PyTorch layer from torch.nn will set these for us.

3. **torch.nn.Module** - The base class for all neural network modules, if you subclass it, you should overwrite forward()

4. **torch.optim** - this where the optimizers in PyTorch live, they will help with gradient descent

5. **def forward()** - All nn.Module subclasses require you to overwrite forward(), this method defines what happens in the forward computation.

In [ ]:
# Create a random seed
torch.manual_seed(42)

# Create an instance of the model (the model is a subclass of nn.Module)
model_0 = LinearRegressionModel()

# Checkout out the parameters
list(model_0.parameters())

In [ ]:
# List names parameters
model_0.state_dict()

### Making predictions using `torch.inference_mode()`

To check our model's predictive power, let's see how well it predicts `y_test` based on `x_test`.

When we pass data through our mode, it's going it through the forward() method.

In [ ]:
x_test

In [ ]:
# Make predictions with model
with torch.inference_mode():
    y_preds = model_0(x_test)
y_preds

In [ ]:
plot_predictions(predictions=y_preds)